Word2vec is a pretrained word embedding provided by Google. The embeddings are trained on the Google News corpus. I will use word2vec to perform sentiment analysis of Amazon product reviews. These reviews were optained from [UCSD](https://colab.research.google.com/drive/1FUTSjPMY5s1u5JXJOPVrPWb8GUfDVQR5#scrollTo=KxrEWktGaJSn&line=1&uniqifier=1)- since the original file was >15 GB, I extracted the first million reviews into a csv and use that directly in this analysis.

In [1]:
#Unzipping data
!unzip amazon_review_mil.csv.zip
reviews_data = 'amazon_review_mil.csv'

Archive:  amazon_review_mil.csv.zip
  inflating: amazon_review_mil.csv   
  inflating: __MACOSX/._amazon_review_mil.csv  


In [25]:
#Importing Libraries

#Machine learning and deep learning
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from nltk.corpus import stopwords
from sklearn.model_selection import train_test_split
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize
from string import punctuation
from gensim.models import Word2Vec, KeyedVectors
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

#pre-processing of text
import string
import re
import json
import gzip
from urllib.request import urlopen
import missingno as mn

#Download stopwords
nltk.download('stopwords')

#basic imports
import os
from time import time


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
#Loading data into pandas dataframe
COLUMNS = ['reviewerID','asin','helpful','reviewText','overall','summary','unixReviewTime','reviewTime','reviewerName','sentiment']
df = pd.read_csv(reviews_data, names = COLUMNS)
df.head()

,reviewerID,asin,helpful,reviewText,overall,summary,unixReviewTime,reviewTime,reviewerName,sentiment
0,A3AF8FFZAZYNE5,0000000078,"[1, 1]",Conversations with God Book 1 is the single mo...,5.0,Impactful!,1092182400,"08 11, 2004",NaN,1
1,AH2L9G3DQHHAJ,0000000116,"[5, 5]",Interesting Grisham tale of a lawyer that take...,4.0,Show me the money!,1019865600,"04 27, 2002",chris,1
2,A2IIIDRK3PRRZY,0000000116,"[0, 0]",The thumbnail is a shirt. The product shown i...,1.0,Listing is all screwed up,1395619200,"03 24, 2014",Helene,-1
3,A1TADCM7YWPQ8M,0000000868,"[10, 10]",I'll be honest. I work for a large online reta...,4.0,Not a Bad Translation,1031702400,"09 11, 2002",Joel@AWS,1
4,AWGH7V0BDOJKB,0000013714,"[0, 0]",It had all the songs I wanted but I had ordere...,4.0,Not the large print,1383177600,"10 31, 2013",Barbara Marshall,1


In [23]:
#Inspecting missing/null values 
print('Number of missing values in each column: ', df.isna().sum())

#Clean missing values
df.dropna(subset = ['reviewText'], inplace = True)


Number of missing values in each column:  reviewerID            0
asin                  0
helpful               0
reviewText          157
overall               0
summary              16
unixReviewTime        0
reviewTime            0
reviewerName      59960
sentiment             0
dtype: int64


In [26]:
#Cleaning the reviews' text
stopwords = set(stopwords.words("english"))
def clean(reviews):
    def remove_stops_digits(doc):
        doc = "".join([char for char in doc if char not in string.punctuation and not char.isdigit()])
        doc = " ".join([token for token in doc.split() if token not in stopwords])
        return doc
    return [remove_stops_digits(review) for review in reviews]


Before training the sentiment analysis classifier, I need to  load and process the word2vec embeddings. So let's do that.

In [14]:
!wget https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM

--2020-08-02 18:36:24--  https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM
Resolving drive.google.com (drive.google.com)... 172.217.212.100, 172.217.212.101, 172.217.212.113, ...
Connecting to drive.google.com (drive.google.com)|172.217.212.100|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/ [following]
--2020-08-02 18:36:24--  https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit [following]
--2020-08-02 18:36:24--  https://drive.google.com/file/d/0B7XkCwpI5KDYNlNUTTlSS21pQmM/edit
Reusing existing connection to drive.google.com:443.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘0B7XkCwpI5KDYNlNUTTlSS21pQmM’

0B7XkCwpI5KDYNlNUTT    

In [27]:
#Load W2V model.
w2v_model = KeyedVectors.load_word2vec_format('/content/word2vec_model', binary=True)
print('done loading Word2Vec')

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:254: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


ValueError: ignored